In [1]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    import sklearn
from WikiCatUtils import (
    Cache,
    read_categories,
    get_fullpath,
    resample_to_equal)
from os.path import exists
from os import mkdir
import numpy as np

if not exists('./test'):                                                               
    mkdir('./test')                                                                    
categories_to_download = read_categories(get_fullpath('example_cats.txt'))                 
cache = Cache(get_fullpath('test/cache'), verbosity=0)                            
for category_uri in categories_to_download:                                                  
    cache.loadCategory(category_uri, only_use_cached=True)# maxlinks=100)


dset, label_map = cache.get_dataset(0.6, 0.2, 0.2)

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility
  return f(*args, **kwds)


In [2]:
# dataset statistics
print({split:len(dset[split][0]) for split in dset})
reverse_label_map = {label_map[category]:category for category in label_map}
indices = reverse_label_map.keys()
print(len(cache.contents))
print("\t%s" % ('\t'.join(map(str,indices))))
for split in dset:
    category_percentages = []
    for category_ind in indices:
        category_percentages.append(dset[split][1].count(category_ind)
                                    / len(dset[split][1]))
    print("%s\t%s" % (split, '\t'.join(map(lambda x: "%0.2f" % x, category_percentages))))

print(reverse_label_map)

{'val': 214, 'train': 635, 'test': 215}
1064
	0	1	2	3	4	5	6
val	0.06	0.03	0.10	0.05	0.03	0.64	0.10
train	0.06	0.03	0.10	0.05	0.03	0.64	0.10
test	0.06	0.03	0.10	0.05	0.03	0.63	0.10
{0: 'Category:Medical devices', 1: 'Category:Organs (anatomy)', 2: 'Category:Congenital disorders', 3: 'Category:Machine learning algorithms', 4: 'Category:Cancer', 5: 'Category:Rare diseases', 6: 'Category:Infectious diseases'}


In [3]:
# this is extremely unbalanced, but we'll see how well it does despite that
# let's try BoW first as a baseline

import re
from string import punctuation
punct_matcher = re.compile(r'[{}]+'.format(re.escape(punctuation)))
space_matcher = re.compile(r'\s+')
def tokenize(article, lower=False):
    # instead of tokenizing by anything fancy we'll just separate by spaces
    # and remove punctuation / quotes / hyphens
    stripped = article.rstrip().lstrip()
    no_punct = re.sub(punct_matcher, '', stripped)
    tokenized = re.split(space_matcher, no_punct)
    if lower:
        tokenized = list(map(lambda x: x.lower(), tokenized))
    return tokenized

def make_vocab(articles, lower=False):
    vocab = {}
    for article in articles:
        tokenized_article = tokenize(article, lower=lower)
        for word in tokenized_article:
            if word not in vocab:
                vocab[word] = 1
            else:
                vocab[word] += 1
    return vocab

def convert_classes(labels):
    # most ml packages expect multiclass labels to be one-hot encoded
    n_classes = max(labels) + 1
    label_indices = list(zip(*[(ind, class_ind) for ind, class_ind in enumerate(labels)]))
    label_mat = np.zeros((len(labels), n_classes))
    label_mat[label_indices] = 1
    return label_mat


#train_vocab = make_vocab(dset['train'][0])
#print(len(train_vocab))
class BoW_transformer:
    def __init__(self, minsample=5, tfidf_weights=False, normalize=True):
        self.use_tfidf = tfidf_weights    
        self.minsample = minsample
        self.normalize = normalize
        
    def get_idf(self, articles):
        # for every word in the vocab, get its inverse doc freq
        self.idf = np.zeros((1,self.vocab_size))
        vocab_appears_in = {word:np.zeros(len(articles), dtype=np.int) 
                            for word in self.lookup}
        for ind, article in enumerate(articles):
            for word in tokenize(article, lower=True):
                if word in self.lookup:
                    vocab_appears_in[word][ind] = 1
        for word in self.lookup:
            self.idf[:,self.lookup[word]] = np.log(1 + len(articles) / np.sum(
                                                vocab_appears_in[word]))
            
    def fit(self, articles, labels):
        # figure out the vocab and tfidf stuff
        self.vocab = make_vocab(articles)
        vocab_sorted = sorted(self.vocab.keys())
        vocab_minsampled = list(filter(lambda x: self.vocab[x] >= self.minsample, 
                                       vocab_sorted))
        self.lookup = {word:ind for ind, word in enumerate(vocab_minsampled)}
        self.vocab_size = len(self.lookup)
        if self.use_tfidf:
            self.get_idf(articles)
        if self.normalize:
            # determine normalization constants
            self.normalize = False
            transformed_articles = self.transform(articles)
            self.normalize = True
            self.maxes = np.max(transformed_articles, axis=0)
            self.mins = np.min(transformed_articles, axis=0)
        return self # needed for pipeline
    
    def transform_single(self, tokenized_article):
        bowvec = np.zeros((1,self.vocab_size), dtype=np.float32)
        for word in tokenized_article:
            if word in self.lookup:
                word_ind = self.lookup[word]
                if self.use_tfidf:
                    bowvec[:,word_ind] += 1
                else:
                    bowvec[:,word_ind] = 1
        if self.use_tfidf:
            # implement the log-scaled frequency
            bowvec = np.log(1 + bowvec)
            bowvec *= self.idf
        if self.normalize:
            bowvec = (bowvec - self.mins) / (self.maxes - self.mins)
            bowvec[np.where(np.isnan(bowvec))] = 0.5 # shouldn't happen but just in case
        return bowvec
    
    def transform(self, articles):
        return np.vstack([self.transform_single(tokenize(article)) 
                          for article in articles])
                
bow_maker = BoW_transformer()
bow_maker.fit(dset['train'][0], dset['train'][1])
train_bow = bow_maker.transform(dset['train'][0])
train_labels = convert_classes(dset['train'][1])
print(train_bow.shape)

(635, 10939)


In [4]:
#print(tokenize(dset['train'][0][0]))
#print(dset['train'][0][0])

In [5]:
from sklearn.linear_model import LogisticRegression
bow_linear = LogisticRegression()
bow_linear.fit(train_bow, dset['train'][1])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [6]:
print(bow_linear.score(train_bow, dset['train'][1]))
# This result (~100% accuracy) shows the inherent issue with having a ton of features
# and very little data -- obviously we're well beyond LogisticRegression's VC dimension
print(np.histogram(bow_linear.predict(train_bow), bins=train_labels.shape[1])[0])
# this is real bad

1.0
[ 36  18  63  31  20 406  61]


In [7]:
# We can very easily see the overfitting here
val_bow = bow_maker.transform(dset['val'][0])
val_labels = convert_classes(dset['val'][1])

print(np.histogram(bow_linear.predict(val_bow), bins=val_labels.shape[1])[0])
print(np.histogram(np.argmax(val_labels, axis=1), bins=val_labels.shape[1])[0])
print(bow_linear.score(val_bow, dset['val'][1]))

[  9   5   9  11   1 157  22]
[ 12   6  21  11   7 136  21]
0.85046728972


In [8]:
# Despite the overfitting it's worth noting that it performs rather well
# However it should also be noted that it is going to be heavily biased towards the 
# 'Infectious Diseases' class since it has far more articles
# Let's see how this holds up under cross validation
from sklearn.cross_validation import cross_val_score
cv_scores = cross_val_score(LogisticRegression(), train_bow, dset['train'][1], cv=5)
print(np.average(cv_scores))
print(np.std(cv_scores))

0.828325491375
0.0341412878191


In [9]:
# So it seems to be getting ~84% accuracy pretty regularly. We'll see if we can improve
# it using TF-IDF weights -- however we need to be careful since we need to learn these 
# weights only on the training set.

In [10]:
bow_tfidf_maker = BoW_transformer(tfidf_weights=True)
bow_tfidf_maker.fit(dset['train'][0], dset['train'][1])
train_bow_tfidf = bow_tfidf_maker.transform(dset['train'][0])


bow_linear_tfidf = LogisticRegression()
bow_linear_tfidf.fit(train_bow_tfidf, dset['train'][1])

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in long_scalars
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in multiply


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [11]:
bow_linear_tfidf.score(train_bow_tfidf, dset['train'][1])

0.99527559055118109

In [12]:
# it doesn't seem like TF-IDF helps, but let's verify that with cross validation
val_bow_tfidf = bow_tfidf_maker.transform(dset['val'][0])
bow_linear_tfidf.score(val_bow_tfidf, dset['val'][1])

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in multiply


0.78971962616822433

In [13]:
# just to be sure we'll make a simple pipeliner that can take the representer and learner
# in one so we can try out CV on this
from sklearn.pipeline import Pipeline
# minsample =~ 5 seems to do the best
bow_tfidf = BoW_transformer(tfidf_weights=True, minsample=5)
bow_logreg = LogisticRegression()
bow_tfidf_logreg = Pipeline([('bow_tfidf', bow_tfidf), ('logreg', bow_logreg)])
bow_tfidf_logreg.fit(dset['train'][0], dset['train'][1])

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in long_scalars
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in multiply


Pipeline(steps=[('bow_tfidf', <__main__.BoW_transformer object at 0x7f54551abfd0>), ('logreg', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [14]:
# Well it does worse
cv_scores = cross_val_score(bow_tfidf_logreg, dset['train'][0], dset['train'][1], cv=5)
print(np.average(cv_scores))
print(np.std(cv_scores))

/usr/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in long_scalars
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in multiply
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in long_scalars
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in multiply
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in long_scalars
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in multiply
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: divide by zero encountered in long_scalars
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:90: RuntimeWarning: invalid value encountered in multiply
/usr/lib/python3.5/site-packages/ipykernel/__main__.py:57: RuntimeWarning: d

0.79201788784
0.0222314405877


In [15]:
# I'm willing to bet that a large part of this is the difference in vocabulary between 
# any given split of the data. To test this
train_vocab = make_vocab(dset['train'][0])
val_vocab = make_vocab(dset['val'][0])
print("Percentage of words in val vocab that are also in train vocab: %0.2f" % 
      (100*len([val_word for val_word in val_vocab.keys() if val_word in train_vocab]) / 
       len(val_vocab)))
# Thus ~35% of the time a word was processed in the validation data, it was not in the
# the training vocab & therefore was skipped, which is pretty awful -- especially when
# we have a strong prior belief that words unique to an article and their latent meaning
# will strongly indicate the category of the article

# We could alleviate this problem at first by using the entire dataset at hand to learn a
# vocabulary, but as we can see below the size of this would be prohibitive (at least 400k)
# We could do something like a sparse 

Percentage of words in val vocab that are also in train vocab: 60.17


In [16]:
# GloVe classifier
# Averaging co-occurence based word embeddings is a pretty powerful baseline 
# in my experience, so let's try it out here using GloVe embeddings
# Conveniently Jeffrey Pennington has trained GloVe embeddings on a 2014 dump of Wikipedia
# which is probably fine for this task. In the interests of expediency I'm not going to 
# train them myself (it will take a long time for lower quality embeddings), but use the
# pretrained embeddings from here: http://nlp.stanford.edu/data/glove.6B.zip
# Since this is on the entirety of Wikipedia, we can safely assume that there is some
# data snooping based on learning these cooccurence statistics
# in addition, this means that it has a larger vocabulary to work with than BoW, alle
# WikiCatBuild will automatically download these into the cache if they're not there
import codecs
def glove_read(dimsize, articles):
    fn = './glove.6B.%dd.txt' % dimsize
    if not exists(fn):
        print("GloVe vectors of size %d aren't there" % dimsize)
        return None
    # we're going to ignore words that aren't in the articles given to reduce memory usage
    vocab_dataset = make_vocab(articles)
    vocab_ret = {}
    with codecs.open(fn, 'r', 'utf8') as f:
        for line in f:
            split_line = str(line).rstrip().split(' ')
            word = split_line[0]
            if word in vocab_dataset:
                vocab_ret[word] = np.array(list(map(float, split_line[1:])))
    return vocab_ret



In [17]:
from collections import defaultdict
class WordEmbedding_transform:
    def __init__(self, vocab, dim, use_tfidf_weights=False, normalize=True):
        # Assume vocab is the mapping of word to ndarray vector
        self.vocab = vocab
        self.dim = dim
        self.use_tfidf = use_tfidf_weights
        self.normalize = normalize
        
    def get_idf(self, articles):
        # for every word in the vocab, get its inverse doc freq
        # essentially this way we avoid divide by zero since if we hit
        # the default we're saying that the word as far as we know only
        # exists in the document we're evaluating
        self.idf = defaultdict(lambda: np.log(1 + len(articles)))
        #local_vocab = make_vocab(articles)
        vocab_appears_in = defaultdict(lambda: np.zeros(len(articles)))
        for ind, article in enumerate(articles):
            for word in tokenize(article):
                if word in self.vocab:
                    vocab_appears_in[word][ind] = 1
        for word in vocab_appears_in:
            self.idf[word] = np.log(1 + len(articles) / 
                                     (np.sum(vocab_appears_in[word])))
            # this should never hit a divide by zero
    
    def fit(self, articles, labels):
        # calculate idf 
        if self.use_tfidf:
            self.get_idf(articles)
        if self.normalize:
            self.normalize = False
            transformed_articles = self.transform(articles)
            self.normalize = True
            self.maxes = np.max(transformed_articles, axis=0)
            self.mins = np.min(transformed_articles, axis=0)
        return self
    
    def transform(self, articles):
        vecs = []
        oov = 0
        total_words = 0
        for article in articles:
            avg_vec = np.zeros((1,self.dim))
            # compute the tfidf weights, req 2 passes unfortunately
            tokenized_article = list(filter(lambda x: x in self.vocab, tokenize(article)))
            word_weight = defaultdict(lambda: 1/len(tokenized_article))
            # word_weight defaults to 1/n for non-tfidf case
            if self.use_tfidf:
                for word in tokenized_article:
                    word_weight[word] = word_weight[word] + int(word in word_weight)
                for word in word_weight:
                    word_weight[word] = np.log(1 + word_weight[word])
                    word_weight[word] *= self.idf[word]
            for word in tokenized_article:
                avg_vec += word_weight[word] * self.vocab[word].reshape(1,self.dim)
            if self.normalize:
                avg_vec = (avg_vec - self.mins) / (self.maxes - self.mins)
                avg_vec[np.where(np.isnan(avg_vec))] = 0.5 # shouldn't happen but just in case
            vecs.append(avg_vec)
        
        return np.vstack(vecs)
    

In [27]:
# Let's determine the best dimensionality to use by cross validation
from itertools import chain
all_articles = list(chain(dset['train'][0], dset['val'][0], dset['test'][0]))
for dim in [50,100,200,300]:
    glove_vecs = glove_read(dim, all_articles)    
    embedder = WordEmbedding_transform(glove_vecs, dim, normalize=True)
    embedder.fit(dset['train'][0], dset['train'][1])
    train = embedder.transform(dset['train'][0])        

    logreg = LogisticRegression()
    logreg.fit(train, dset['train'][1])
    

    print("%d: Training acc: %0.3f" % (dim, logreg.score(train, dset['train'][1])))
    
    cv_scores = cross_val_score(logreg, train, dset['train'][1], n_jobs=8, cv=8)

    print("%d: CV acc: %0.3f w/std %0.3f" % (dim, 
                                             np.average(cv_scores), 
                                             np.std(cv_scores)))
    
    tfidf_emb = WordEmbedding_transform(glove_vecs, dim, use_tfidf_weights=True, normalize=True)
    tfidf_logreg = Pipeline([('tfidf_emb', tfidf_emb), ('logreg', logreg)])
    tfidf_logreg.fit(dset['train'][0], dset['train'][1])
    tfidf_cv_scores = cross_val_score(tfidf_logreg,
                                      dset['train'][0],
                                      dset['train'][1], cv=8)

    print("%d: TFIDF Training acc: %0.3f" % (dim, tfidf_logreg.score(dset['train'][0],
                                                                     dset['train'][1])))
    print("%d: TFIDF CV acc: %0.3f w/std %0.3f" % (dim, 
                                             np.average(tfidf_cv_scores), 
                                             np.std(tfidf_cv_scores)))
    #val = embedder.transform(dset['val'][0])
    #print("Validation acc: %0.3f" % logreg.score(val, dset['val'][1]))
    


50: Training acc: 0.802
50: CV acc: 0.777 w/std 0.015
50: TFIDF Training acc: 0.666
50: TFIDF CV acc: 0.662 w/std 0.023
100: Training acc: 0.843
100: CV acc: 0.805 w/std 0.022
100: TFIDF Training acc: 0.683
100: TFIDF CV acc: 0.676 w/std 0.023
200: Training acc: 0.880
200: CV acc: 0.819 w/std 0.034
200: TFIDF Training acc: 0.698
200: TFIDF CV acc: 0.686 w/std 0.020
300: Training acc: 0.906
300: CV acc: 0.832 w/std 0.033
300: TFIDF Training acc: 0.715
300: TFIDF CV acc: 0.692 w/std 0.021


In [29]:
# I've re-run the above several times with different splits of the dataset and found (mostly)
# that 200 is the best dimensionality, normalization hurts performance but helps control
# overfitting when using TF-IDF, though this normalization is super necessary for good
# performance without TF-IDF.

# Initially my thoughts were that TF-IDF weights would help, but they seem to cause heavy
# overfitting. My theory is that only learning the TF-IDF weights on the training data is
# the cause of this, and in retrospect this doesn't really seem justified since I'm using
# GloVe embeddings that were trained on the entire Wikipedia.
# Instead of determining TF-IDF weights from the whole Wikipedia, let's just use the entire
# dataset (or in this case training + val)
tfidf_g200 = glove_read(200, all_articles)
tfidf_emb = WordEmbedding_transform(tfidf_g200, 200, normalize=False, use_tfidf_weights=True)
tfidf_emb.fit(list(chain(dset['train'][0], dset['val'][0])),
              list(chain(dset['train'][1], dset['val'][1])))
tfidf_train = tfidf_emb.transform(dset['train'][0])
tfidf_val = tfidf_emb.transform(dset['val'][0])
tfidf_logreg = LogisticRegression(C=0.125)
tfidf_logreg.fit(tfidf_train, dset['train'][1])
print(tfidf_logreg.score(tfidf_train, dset['train'][1]))
print(tfidf_logreg.score(tfidf_val, dset['val'][1]))
# It would appear that the overfitting isn't helped by learning the TF-IDF weights from the
# whole corpus, but normalizing the results does help prevent overfitting (at the cost of
# reducing performance overall)

0.957480314961
0.757009345794


In [24]:
# I think based solely on this limited and most likely incomplete analysis I'll go with
# keeping the representation at averaging 200-dimensional pre-trained GloVe vectors, and
# normalizing them.

# Now let's try a few different classifiers to see what makes sense
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

# For the initial analysis the time expenditure that would be needed to find the best
# hyperparameters is a bit out of scope, so we'll use the sklearn defaults

best_emb = WordEmbedding_transform(tfidf_g200, 200, normalize=True)
best_emb.fit(dset['train'][0], dset['train'][1])
train_data = best_emb.transform(dset['train'][0])
val_data = best_emb.transform(dset['val'][0])
emb_svm = LinearSVC(class_weight='auto')
emb_rf = RandomForestClassifier(class_weight='auto')
emb_knn = KNeighborsClassifier()
emb_lr = LogisticRegression(class_weight='auto')

classifiers = [emb_svm, emb_rf, emb_knn]#, emb_lr]
for classifier in classifiers:
    print("Evaluating %s" % classifier.__class__)
    classifier.fit(train_data, dset['train'][1])
    train_score = classifier.score(train_data, dset['train'][1])
    cv_scores = cross_val_score(classifier, train_data, dset['train'][1], cv=10)
    cv_avg = np.average(cv_scores)
    cv_std = np.std(cv_scores)
    val_score = classifier.score(val_data, dset['val'][1])
    print("\tTraining accuracy: %0.2f" % train_score)
    print("\tCV score: %0.2f (+/- %0.2f)" % (cv_avg, cv_std/2))
    print("\tVal score: %0.2f" % val_score)

# So interestingly the SVM overfits very heavily but gets a better validation accuracy
# for it, though the high variance in its CV scores are concerning. RF overfits very
# heavily, 

Evaluating <class 'sklearn.svm.classes.LinearSVC'>
	Training accuracy: 0.92
	CV score: 0.80 (+/- 0.01)
	Val score: 0.80
Evaluating <class 'sklearn.ensemble.forest.RandomForestClassifier'>
	Training accuracy: 0.99
	CV score: 0.77 (+/- 0.01)
	Val score: 0.75
Evaluating <class 'sklearn.neighbors.classification.KNeighborsClassifier'>
	Training accuracy: 0.84
	CV score: 0.80 (+/- 0.01)
	Val score: 0.78


In [25]:
# So K-NN actually seems to be the most stable / overfit the least, however I won't use it
# because it's not at all scalable. Besides that, it appears that LogisticRegression is 
# the way to go. 
# Notably this is using the 'One-v-Rest' strategy for training (I tried multinomial, 
# results were significantly worse).

# Let's try regularizing it now w/L2 regularization and looking over the possible C values
# Powers of 10 shows that between C = 1e-1 and C=1e0 works
# Binary search down to 0.125
# Ultimately it looks like C = 0.125 gives the best C score w/minimal overfitting
emb_lr.C = 0.125
emb_lr.fit(train_data, dset['train'][1])
train_score = emb_lr.score(train_data, dset['train'][1])
cv_scores = cross_val_score(emb_lr, train_data, dset['train'][1], cv=10)
cv_avg = np.average(cv_scores)
cv_std = np.std(cv_scores)
val_score = classifier.score(val_data, dset['val'][1])
print("Training accuracy: %0.2f" % train_score)
print("CV score: %0.2f (+/- %0.2f)" % (cv_avg, cv_std/2))
print("Val score: %0.2f" % val_score)

print(np.linalg.norm(emb_lr.coef_, axis=1))

Training accuracy: 0.85
CV score: 0.82 (+/- 0.01)
Val score: 0.78
[ 2.34172859  2.58260842  1.36911414  2.51661591  2.28602989  1.34230777
  2.26507416]


-0.0049886622043486279